In [ ]:
#pip install --user git+https://github.com/brazil-data-cube/EOCubes@master --upgrade --no-cache-dir

In [ ]:
### Import eocubes ###
from bdc_eocubes.business import Business as eocubes

In [ ]:
### List cubes ###
cubes = eocubes.list_cubes()
cubes

In [ ]:
### Get cube description ###
cube_s10 = eocubes.describe_cube('S10m:STACK')
cube_s10

In [ ]:
### get cube ###
my_cube = eocubes.get_cube('S10m:STACK', bbox= '-46.62597656250001,-13.19716452328198,-45.03570556640626,-12.297068292853805')
my_cube

In [ ]:
bands = my_cube['2018-09-01/2019-08-01']['tiles']['089098']['bands'].keys()
print( bands )

In [ ]:
links_nir = my_cube['2018-09-01/2019-08-01']['tiles']['089098']['bands']['nir']
links_nir

In [ ]:
filename_nir = links_nir[0]
filename_nir

In [ ]:
#### Import Libs ####
import numpy
import rasterio

from math import floor, ceil
from matplotlib import pyplot as plt
from pyproj import Proj
from rasterio.warp import transform
from rasterio.windows import Window

In [ ]:
def longlat2window(lon, lat, dataset):
    """
    Args:
        lon (tuple): Tuple of min and max lon
        lat (tuple): Tuple of min and max lat
        dataset: Rasterio dataset

    Returns:
        rasterio.windows.Window
    """
    p = Proj(dataset.crs)
    t = dataset.transform
    xmin, ymin = p(lon[0], lat[0])
    xmax, ymax = p(lon[1], lat[1])
    col_min, row_min = ~t * (xmin, ymin)
    col_max, row_max = ~t * (xmax, ymax)
    return Window.from_slices(rows=(floor(row_max), ceil(row_min)),
                              cols=(floor(col_min), ceil(col_max)))

In [ ]:
with rasterio.open(filename_nir) as dataset:
    print(dataset.profile)
    #img = dataset.read(1) ### open entire scene is heavy
    
    img = dataset.read(1, window=Window(0, 0, 3000, 3000)) ### Window(col_off, row_off, width, height)
    
    plt.imshow(img, cmap='gray')
    plt.show()

In [ ]:
### CREATING BBOX ###
w = -45.90
n = -12.6
e = -45.40
s = -12.90

with rasterio.open(filename_nir) as dataset:
    img = dataset.read(1, window = longlat2window((w,e), (s,n), dataset)) ### Window(col_off, row_off, width, height)

    plt.imshow(img, cmap='gray')
    plt.show()

In [ ]:
### Open several bands

In [ ]:
print( bands )

In [ ]:
### select a few bands
my_bands = ('blue', 'green', 'red', 'nir')
my_bands

In [ ]:
import xarray

nir = xarray.open_rasterio(filename_nir)

In [ ]:
nir.variable.data

In [ ]:
cube = my_cube['2018-09-01/2019-08-01']['tiles']['089098']['bands']
for band in my_bands:
    print( band )
    
    for link in cube[band]:
        print( link )
    #load bands to xarray dataset

In [ ]:
### LEAFLET

In [ ]:
from ipyleaflet import Map, WMSLayer

In [ ]:
m = Map(center=[-12.75, -45.65], zoom=10)
m

In [ ]:
#sh_wms_url = 'https://services.sentinel-hub.com/ogc/wms/' + INSTANCE_ID + '?showlogo=0&time=2017-07-18/2017-07-18'
#sh_wms_url

In [ ]:
#m.add_layer(WMSLayer(url=sh_wms_url, layers="GEOLOGY", tile_size=512))
#print(m)

In [ ]:
#center coordinates resulting in a bit off-center
m.zoom = 8
m.center=[-12.6, -46] 

In [ ]:
m.bounds